TODO in this code:
1. Properly clean the values generated by the weekly plan and populate them using filtered workouts dataset.(done)
2. Assign number of sets of workouts and durations.
3. Create health conditions rules for duration.
Optional but should do this:
4. Create a ratio of warm-up,main and cool-down exercises.

Issues:
1. Core+lower body workout is not being shown for weight loss workout(kinda fixed)
2. Weight loss should have had two Full Body HIIT workouts in a week but that is not happening.(Fixed)
3. Active rest day algorithm is failing.(fixed)
4. Since the week starts with Monday, if the rest day is chosen to be sunday then the active day functionality will not be implemented until first complete day passes.(Fixed)

Validation:
Cross-check workouts and intensities for different scenarios and create a validation table.

In [ ]:
#FOR GUJJAR' ATTENTION !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#DO NOT IGNORE THESE ATTENTION SEEKING COMMENTS IN THE CODE


#This is not a part of the code treat this as a document please
#Upper body strength: about 40 ex, muscle groups:(biceps, triceps, chest, shoulders, lats?), Type:(strength)
#Lower Body: about 44 ex, muscle groups:(abductors, quadriceps, glutes, hamstrings, calves, adductors), Type:(strength)
#Core and Abs Workout: about 171 ex, muscle groups:(abdominals, middle back, chest), Type: (strength, core)
#Light Cardio: about 38 ex, Type:(cardio, mobility), Met Values(3.5-6)
#Core + Chest: about 112
#Full body Strength: about 80, Type:(Strength)
#Cardio Endurance: about 38
#Full body HIIT: about 38 almost same as cardio

In [3]:
!pip install pydatalog
from pyDatalog import pyDatalog

# Initialize terms
pyDatalog.create_terms('Age, Activity, MetRange, Difficulty, Min, Max')

# Define age group ranges
Min['adult'] = 15
Max['adult'] = 34
Min['middle_aged'] = 35
Max['middle_aged'] = 49
Min['older_adult'] = 50
Max['older_adult'] = 120

# MET range labels (string output, not numerical)
pyDatalog.create_terms('recommended_met, recommended_difficulty')

# Consistently lowercase difficulty terms
beginner = 'beginner'
intermediate = 'intermediate'

# --- ADULTS (15-34) ---
recommended_met(Age, 'low', 'low') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'moderate', 'low') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'moderate', 'moderate') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'high', 'low') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'high', 'moderate') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'high', 'high') <= (Min['adult'] <= Age) & (Age <= Max['adult'])

recommended_difficulty(Age, 'low', beginner) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'low', intermediate) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'high', beginner) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'high', intermediate) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'moderate', beginner) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'moderate', intermediate) <= (Min['adult'] <= Age) & (Age <= Max['adult'])

# --- MIDDLE-AGED (35–49) ---
recommended_met(Age, Activity, 'low') <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged']) & (Activity != 'high')
recommended_met(Age, 'high', 'low') <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])
recommended_met(Age, 'high', 'moderate') <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])

recommended_difficulty(Age, Activity, beginner) <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])
recommended_difficulty(Age, 'high', intermediate) <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])

# --- OLDER ADULTS (50+) ---
recommended_met(Age, Activity, 'low') <= (Age >= Min['older_adult']) & (Activity != 'high')
recommended_met(Age, 'high', 'low') <= (Age >= Min['older_adult'])
recommended_met(Age, 'high', 'moderate') <= (Age >= Min['older_adult'])

recommended_difficulty(Age, Activity, beginner) <= (Age >= Min['older_adult'])


#Example Queries
#print(recommended_met(18, 'low', MetRange).data)         # [('low',)]
#print(recommended_difficulty(18, 'low', Difficulty).data)  # [('beginner',), ('intermediate',)]
#print(recommended_met(40, 'moderate', MetRange).data)    # [('low',)]
#print(recommended_difficulty(40, 'moderate', Difficulty).data)  # [('beginner',)]
#print(recommended_met(60, 'high', MetRange).data)        # [('low',), ('moderate',)]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pydatalog: filename=pydatalog-0.17.4-cp311-cp311-linux_x86_64.whl size=1549126 sha256=556fa93a678551ab8819dfc04b9cc4e787a6e6c2095f590769272f78aac1e9a4
  Stored in directory: /root/.cache/pip/wheels/8f/f8/a0/c12514fe74e5f69d0d8967bd92363ea7a2961589ead1350b1d
Successfully built pydatalog


recommended_difficulty(Age,Activity,'beginner') <=

In [4]:
import pandas as pd

#Cleaning the diffiuclty column for trailing whitespaces
df = pd.read_csv("workouts.csv")
df['Difficulty'] = df['Difficulty'].str.strip().str.capitalize()

# MET Ranges mapping
MET_RANGES = {
    'low': (1.5, 3.9),
    'moderate': (4.0, 6.9),
    'high': (7.0, 10.0)
}

def get_user_input():
    return {
        'age': 18,
        'activity': 'high'
    }

def get_recommendations(age, activity):
    met_output = recommended_met(age, activity, MetRange).data
    diff_output = recommended_difficulty(age, activity, Difficulty).data

    met_recs = [item[0] for item in met_output]
    diff_recs = [item[0].capitalize() for item in diff_output]

    return met_recs, diff_recs

def filter_workouts(df, met_recs, diff_recs):
    met_mask = pd.Series([False] * len(df))
    for met_level in met_recs:
        if met_level in MET_RANGES:
            min_val, max_val = MET_RANGES[met_level]
            met_mask |= df['MET Value'].between(min_val, max_val)

    filtered_df = df[met_mask & df['Difficulty'].isin(diff_recs)]
    return filtered_df


def main():
    user = get_user_input()
    met_recs, diff_recs = get_recommendations(user['age'], user['activity'])

    print("Recommended MET Levels:", met_recs)
    print("Recommended Difficulties:", diff_recs)

    personalized_df = filter_workouts(df, met_recs, diff_recs)

    if not personalized_df.empty:
        print("\n Personalized Workout Recommendations:\n")
        print(personalized_df)
    else:
        print("\n No workouts match the given filters.")

    personalized_df.to_csv("filtered_workouts.csv", index=False)
main()


Recommended MET Levels: ['high', 'moderate', 'low']
Recommended Difficulties: ['Intermediate', 'Beginner']

 Personalized Workout Recommendations:

                            Name Target Muscle        Type  MET Value  \
0          Child's pose Stretch    Middle Back  Stretching        2.0   
1    Front and Back Neck Stretch   Middle Back  Stretching        2.0   
2           Lateral Neck Stretch   Middle Back  Stretching        2.0   
3               Shoulder Circles    Quadriceps  Stretching        2.0   
4                  Ankle Circles        Calves    Mobility        2.3   
..                           ...           ...         ...        ...   
395                       Burpee     Full body      Cardio        8.0   
396                   High Knees        Glutes      Cardio        8.0   
397                      Frogger        Glutes      Cardio        8.0   
400                    Star Jump    Quadriceps      Cardio        8.0   
401                 Jumping Jack    Quadriceps   

In [11]:
#Determining the focus for every day as well as calculating and assigning active and complete rest days
import pandas as pd

#Inputs
PROGRAM_DURATION = 30  # Can be any number i.e 30,90,60,15,10
GOAL = 'weight_loss'  # 'stay_fit', 'gain_weight', 'weight_loss', 'build_muscle'
PREFERRED_REST_DAY = 'Thursday'  # User's preferred rest day (Complete Rest Day) active rest day will be calculated using this complete rest day's position in the week

# constants
DAYS_OF_WEEK = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
GOAL_FOCUS_TEMPLATES = {
    'stay_fit': [
        'Upper Body Strength', 'Lower Body Strength', 'Core + Abs',
        'Light Endurance', 'Light Endurance'
    ],
    'gain_weight': [
        'Upper Body Strength', 'Core + Chest', 'Lower Body Strength',
        'Light Endurance', 'Full Body Strength'
    ],
    'weight_loss': [
        'Cardio', 'Full Body HIIT', 'Core + Lower Body',
        'Full Body HIIT', 'Abs + Upper Body'
    ],
    'build_muscle': [
        'Upper Body Strength', 'Lower Body Strength', 'Upper Body Strength',
        'Core + Abs', 'Cardio'
    ]
}

# Cleaners
def get_weekday(day_index):
    return DAYS_OF_WEEK[day_index % 7]

def get_next_focus(template_list, day_number):
    return template_list[day_number % len(template_list)]

#Main plan generation
def generate_plan(goal, program_duration, preferred_rest_day):
    plan = []
    focus_template = GOAL_FOCUS_TEMPLATES[goal]
    focus_day_counter = 0
    workout_streak = 0  # Count how many workout days since last rest

    for i in range(program_duration):
        weekday = get_weekday(i)

        if weekday == preferred_rest_day:
            focus = 'Complete Rest Day'
            workout_streak = 0  # reset streak after rest
        elif workout_streak >= 3:
            focus = 'Active Rest Day'
            workout_streak = 0
        else:
            focus = get_next_focus(focus_template, focus_day_counter)
            focus_day_counter += 1
            workout_streak += 1

        plan.append({
            'Day': f'Day {i+1} ({weekday})',
            'Focus': focus,
            'Duration': 'TBD'
        })

    return plan

def print_plan(plan):
    for entry in plan:
        print(entry)

def export_to_csv(plan, filename='weekly_plan_with_focus.csv'):
    pd.DataFrame(plan).to_csv(filename, index=False)

weekly_plan = generate_plan(GOAL, PROGRAM_DURATION, PREFERRED_REST_DAY)
print_plan(weekly_plan)
export_to_csv(weekly_plan)

{'Day': 'Day 1 (Monday)', 'Focus': 'Cardio', 'Duration': 'TBD'}
{'Day': 'Day 2 (Tuesday)', 'Focus': 'Full Body HIIT', 'Duration': 'TBD'}
{'Day': 'Day 3 (Wednesday)', 'Focus': 'Core + Lower Body', 'Duration': 'TBD'}
{'Day': 'Day 4 (Thursday)', 'Focus': 'Complete Rest Day', 'Duration': 'TBD'}
{'Day': 'Day 5 (Friday)', 'Focus': 'Full Body HIIT', 'Duration': 'TBD'}
{'Day': 'Day 6 (Saturday)', 'Focus': 'Abs + Upper Body', 'Duration': 'TBD'}
{'Day': 'Day 7 (Sunday)', 'Focus': 'Cardio', 'Duration': 'TBD'}
{'Day': 'Day 8 (Monday)', 'Focus': 'Active Rest Day', 'Duration': 'TBD'}
{'Day': 'Day 9 (Tuesday)', 'Focus': 'Full Body HIIT', 'Duration': 'TBD'}
{'Day': 'Day 10 (Wednesday)', 'Focus': 'Core + Lower Body', 'Duration': 'TBD'}
{'Day': 'Day 11 (Thursday)', 'Focus': 'Complete Rest Day', 'Duration': 'TBD'}
{'Day': 'Day 12 (Friday)', 'Focus': 'Full Body HIIT', 'Duration': 'TBD'}
{'Day': 'Day 13 (Saturday)', 'Focus': 'Abs + Upper Body', 'Duration': 'TBD'}
{'Day': 'Day 14 (Sunday)', 'Focus': 'Cardio

In [13]:
#GUJJJAAARRRR'S ATTENTION!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#This code cell is just populating the days of the weekly plan with a single exercise because i wanted to validate the model. This will produce a "single_populated_workout_plan" file in the CSV Format. Don't confuse this with an actual output workout plan that will be shown to the user but also don't change it FOR NOW AT LEAST
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
import pandas as pd

# Load your filtered workout dataset
workouts_df = pd.read_csv("filtered_workouts.csv")

# Define mappings: focus_area -> target_muscles and type/caution filters
focus_area_definitions = {
    'Upper Body Strength': {
        'target_muscles': ['Forearms', 'Shoulders', 'Biceps', 'Triceps', 'Chest'],
        'Type': ['Strength']
    },
    'Lower Body Strength': {
        'target_muscles': ['Hamstrings', 'Glutes', 'Quadriceps', 'Calves', 'Abductors', 'Adductors'],
        'Type': ['Strength']
    },
    'Core + Abs': {
        'target_muscles': ['Abdominals'],
        'Type': ['Strength', 'Core']
    },
    'Core + Chest': {
        'target_muscles': ['Abdominals', 'Chest'],
        'Type': ['Strength', 'Core']
    },
    'Light Endurance': {
        'target_muscles': ['Quadriceps', 'Calves', 'Glutes', 'Hamstrings'],
        'Type': ['Cardio', 'Mobility']
    },
    'Full Body Strength': {
        'target_muscles': ['Full Body', 'Biceps', 'Triceps', 'Shoulders', 'Glutes', 'Hamstrings'],
        'Type': ['Strength']
    },
    'Full Body HIIT': {
        'target_muscles': [],
        'Type': [],
        'Caution': ['HIIT', 'Plyometric HIIT', 'Isometric HIIT']
    },
    'Core + Lower Body': {
        'target_muscles': ['Abdominals','Hamstrings', 'Glutes', 'Quadriceps', 'Calves', 'Abductors', 'Adductors'],
        'Type': ['Strength', 'Core']
    },
    'Abs + Upper Body': {
        'target_muscles': ['Abdominals', 'Shoulders', 'Biceps', 'Triceps', 'Middle Back', 'Lower Back'],
        'Type': ['Strength']
    },
    'Cardio': {
        'target_muscles': [],
        'Type': ['Cardio']
    },
    'Active Rest Day': {
        'target_muscles': [],
        'Type': ['Mobility', 'Stretching']
    }
}

# Load your weekly plan with focus
plan_df = pd.read_csv("weekly_plan_with_focus.csv")

# Function to filter workouts based on focus area logic
def get_workouts_for_focus(focus_area):
    filters = focus_area_definitions.get(focus_area, {})
    if not filters:
        return []

    # Prepare each filter
    muscle_filter = workouts_df['Target Muscle'].isin(filters.get('target_muscles', [])) if filters.get('target_muscles') else True
    type_filter = workouts_df['Type'].isin(filters.get('Type', [])) if filters.get('Type') else True
    caution_filter = workouts_df['Caution'].isin(filters.get('Caution', [])) if filters.get('Caution') else True

    # Combine filters depending on which are booleans
    if isinstance(muscle_filter, bool):
        filtered = workouts_df[type_filter & caution_filter]
    elif isinstance(type_filter, bool):
        filtered = workouts_df[muscle_filter & caution_filter]
    elif isinstance(caution_filter, bool):
        filtered = workouts_df[muscle_filter & type_filter]
    else:
        filtered = workouts_df[muscle_filter & type_filter & caution_filter]

    # Apply additional logic for Light Endurance (filter difficulty + MET value)
    if focus_area == 'Light Endurance':
        filtered = filtered[
            (filtered['Difficulty'].isin(['Beginner', 'Intermediate'])) &
            (filtered['MET Value'] <= 6)
        ]

    return filtered.sort_values(by='MET Value').to_dict('records')

# Track workout usage for each focus area to avoid repetition
focus_index_tracker = {}
populated_plan = []

# Build the final plan
for _, row in plan_df.iterrows():
    day_entry = dict(row)
    focus = row['Focus']

    if 'Rest' in focus:
        day_entry['Workout Name'] = ''
        day_entry['Target Muscle'] = ''
        day_entry['Workout Type'] = ''
        day_entry['Duration'] = '0 mins'
    else:
        workouts = get_workouts_for_focus(focus)
        current_index = focus_index_tracker.get(focus, 0)

        if workouts:
            workout = workouts[current_index % len(workouts)]
            focus_index_tracker[focus] = current_index + 1

            day_entry['Workout Name'] = workout['Name']
            day_entry['Target Muscle'] = workout['Target Muscle']
            day_entry['Workout Type'] = workout.get('Type', 'TBD')
            day_entry['Duration'] = f"{workout.get('Duration', 'TBD')} mins"
        else:
            day_entry['Workout Name'] = 'No workout available'
            day_entry['Target Muscle'] = ''
            day_entry['Workout Type'] = ''
            day_entry['Duration'] = 'TBD'

    populated_plan.append(day_entry)

# Save the final populated plan
populated_df = pd.DataFrame(populated_plan)
populated_df.to_csv("single_populated_workout_plan.csv", index=False)

print("✅ Final workout plan saved to 'single_populated_workout_plan.csv'")


✅ Final workout plan saved to 'final_populated_workout_plan.csv'


In [12]:
#GUJJJJJAAAARRRRR!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#This is what the actual output workouts plan will look like with more than one workouts stored against a single day. For now i have kept the maximum number of workouts just 3 for a day i will make a model to figure that part out as well. This will return a "final_populated_workout_plan" in JSON format that will be the actual workout plan for a week for a user. USE THIS FOR THE UI INTERFACE.
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
import pandas as pd
import json

# Load your filtered workout dataset
workouts_df = pd.read_csv("filtered_workouts.csv")

# Define mappings: focus_area -> target_muscles and type/caution filters
focus_area_definitions = {
    'Upper Body Strength': {
        'target_muscles': ['Forearms', 'Shoulders', 'Biceps', 'Triceps', 'Chest'],
        'Type': ['Strength']
    },
    'Lower Body Strength': {
        'target_muscles': ['Hamstrings', 'Glutes', 'Quadriceps', 'Calves', 'Abductors', 'Adductors'],
        'Type': ['Strength']
    },
    'Core + Abs': {
        'target_muscles': ['Abdominals'],
        'Type': ['Strength', 'Core']
    },
    'Core + Chest': {
        'target_muscles': ['Abdominals', 'Chest'],
        'Type': ['Strength', 'Core']
    },
    'Light Endurance': {
        'target_muscles': ['Quadriceps', 'Calves', 'Glutes', 'Hamstrings'],
        'Type': ['Cardio', 'Mobility']
    },
    'Full Body Strength': {
        'target_muscles': ['Full Body', 'Biceps', 'Triceps', 'Shoulders', 'Glutes', 'Hamstrings'],
        'Type': ['Strength']
    },
    'Full Body HIIT': {
        'target_muscles': [],
        'Type': [],
        'Caution': ['HIIT', 'Plyometric HIIT', 'Isometric HIIT']
    },
    'Core + Lower Body': {
        'target_muscles': ['Abdominals','Hamstrings', 'Glutes', 'Quadriceps', 'Calves', 'Abductors', 'Adductors'],
        'Type': ['Strength', 'Core']
    },
    'Abs + Upper Body': {
        'target_muscles': ['Abdominals', 'Shoulders', 'Biceps', 'Triceps', 'Middle Back', 'Lower Back'],
        'Type': ['Strength']
    },
    'Cardio': {
        'target_muscles': [],
        'Type': ['Cardio']
    },
    'Active Rest Day': {
        'target_muscles': [],
        'Type': ['Mobility', 'Stretching']
    }
}

# Load your weekly plan with focus
plan_df = pd.read_csv("weekly_plan_with_focus.csv")

# Function to filter workouts based on focus area logic
def get_workouts_for_focus(focus_area):
    filters = focus_area_definitions.get(focus_area, {})
    if not filters:
        return []

    # Prepare each filter
    muscle_filter = workouts_df['Target Muscle'].isin(filters.get('target_muscles', [])) if filters.get('target_muscles') else True
    type_filter = workouts_df['Type'].isin(filters.get('Type', [])) if filters.get('Type') else True
    caution_filter = workouts_df['Caution'].isin(filters.get('Caution', [])) if filters.get('Caution') else True

    # Combine filters depending on which are booleans
    if isinstance(muscle_filter, bool):
        filtered = workouts_df[type_filter & caution_filter]
    elif isinstance(type_filter, bool):
        filtered = workouts_df[muscle_filter & caution_filter]
    elif isinstance(caution_filter, bool):
        filtered = workouts_df[muscle_filter & type_filter]
    else:
        filtered = workouts_df[muscle_filter & type_filter & caution_filter]

    # Apply additional logic for Light Endurance (filter difficulty + MET value)
    if focus_area == 'Light Endurance':
        filtered = filtered[
            (filtered['Difficulty'].isin(['Beginner', 'Intermediate'])) &
            (filtered['MET Value'] <= 6)
        ]

    return filtered.sort_values(by='MET Value').to_dict('records')

# Number of workouts per day
NUM_WORKOUTS_PER_DAY = 3

# Track workout usage for each focus area to avoid repetition
focus_index_tracker = {}
populated_plan = []

# Build the final plan
for _, row in plan_df.iterrows():
    day_entry = dict(row)
    focus = row['Focus']

    if 'Rest' in focus:
        day_entry['Workouts'] = []
    else:
        workouts = get_workouts_for_focus(focus)
        start_index = focus_index_tracker.get(focus, 0)
        end_index = start_index + NUM_WORKOUTS_PER_DAY

        if workouts:
            selected_workouts = workouts[start_index:end_index]

            # If not enough workouts left, wrap around
            if len(selected_workouts) < NUM_WORKOUTS_PER_DAY:
                selected_workouts += workouts[:NUM_WORKOUTS_PER_DAY - len(selected_workouts)]

            focus_index_tracker[focus] = end_index % len(workouts)

            # Save the structured list of workouts
            day_entry['Workouts'] = selected_workouts
        else:
            day_entry['Workouts'] = []

    populated_plan.append(day_entry)

# Save the final populated plan to JSON
with open("final_populated_workout_plan.json", "w") as f:
    json.dump(populated_plan, f, indent=2)

print("✅ Final structured workout plan saved to 'final_populated_workout_plan.json'")


✅ Final structured workout plan saved to 'final_populated_workout_plan.json'
